# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

from sqlalchemy import create_engine
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_response.db')
df = pd.read_sql('SELECT * FROM disaster_response', engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hsg82\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hsg82\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hsg82\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def tokenize(text):
    # normalize text and strip punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    token_list = word_tokenize(text) 
    token_list = [w for w in token_list if w not in stopwords.words("english")]
    
    # lemmatize and return token list
    token_list = [WordNetLemmatizer().lemmatize(w) for w in token_list]
    return token_list

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
import xgboost as xgb

def pipeline_xgbClassifier():
    pipeline = Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('Multi_classifier',MultiOutputClassifier(xgb.XGBClassifier()))
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [12]:
pipeline = pipeline_xgbClassifier()
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred = pipeline.predict(X_test)        

In [14]:
target_name_list = df.columns[4:].tolist()

In [15]:
for i,col in enumerate(target_name_list):
    print(col)
    print(classification_report(y_test[col].values, y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.75      0.14      0.23      2118
           1       0.78      0.99      0.87      6534

   micro avg       0.78      0.78      0.78      8652
   macro avg       0.76      0.56      0.55      8652
weighted avg       0.77      0.78      0.71      8652

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      7180
           1       0.84      0.47      0.60      1472

   micro avg       0.89      0.89      0.89      8652
   macro avg       0.87      0.72      0.77      8652
weighted avg       0.89      0.89      0.88      8652

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8614
           1       0.00      0.00      0.00        38

   micro avg       1.00      1.00      1.00      8652
   macro avg       0.50      0.50      0.50      8652
weighted avg       0.99      1.00      0.99      865

C:\Anaconda3\envs\data_basic\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\data_basic\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda3\envs\data_basic\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8567
           1       0.25      0.02      0.04        85

   micro avg       0.99      0.99      0.99      8652
   macro avg       0.62      0.51      0.52      8652
weighted avg       0.98      0.99      0.99      8652

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8618
           1       0.50      0.03      0.06        34

   micro avg       1.00      1.00      1.00      8652
   macro avg       0.75      0.51      0.53      8652
weighted avg       0.99      1.00      0.99      8652

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8537
           1       0.43      0.03      0.05       115

   micro avg       0.99      0.99      0.99      8652
   macro avg       0.71      0.51      0.52      8652
weighted avg       0.98      0.99      0.98      8652

o

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
parameters =  {
        'vect__max_df': (0.75, 1.0),
        'vect__max_features': (None, 10000),
        'Multi_classifier__estimator__n_estimators' : (90,100,110),
        'Multi_classifier__estimator__learning_rate': (0.1,0.15)}

grid_cv = GridSearchCV(pipeline, parameters, cv = 5,  verbose = 1)

In [19]:
grid_cv.fit(X_train,y_train) 

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.